# T0 Projektinitialisierung und Setup
## Projektziel
Entwicklung eines intelligenten Chatbot-Systems mit:
- Natürlicher Sprachverarbeitung
- Kontextverständnis
- Antwortgenerierung
- Dialogmanagement

## T0.0 Installation und Import der benötigten Bibliotheken
Installation und Import aller notwendigen Pakete für das Projekt.

In [ ]:
# Grundlegende Bibliotheken - Installation
!pip install transformers torch numpy pandas scikit-learn
!pip install --quiet transformers
!pip install --quiet torch
!pip install --quiet numpy pandas scikit-learn
!pip install nltk spacy gensim
!pip install --quiet nltk spacy
!pip install flask python-dotenv
!pip install --quiet flask python-dotenv

# Überprüfung der Installationen
import pkg_resources
def check_package(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        print(f"{package_name}: v{version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package_name}: Nicht installiert!")

for package in ['transformers', 'torch', 'numpy', 'pandas', 'scikit-learn', 'nltk', 'spacy', 'flask']:
    check_package(package)

# Grundlegende Bibliotheken - Imports
try:
    import torch
    import transformers
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    import nltk
    import spacy
    import logging
    import os
    from dotenv import load_dotenv

    print("Alle Module erfolgreich importiert!")
except ImportError as e:
    print(f"Fehler beim Importieren: {e}")
    raise

# Initialisierung der Logging-Konfiguration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Download notwendiger NLTK-Daten mit Fehlerbehandlung
try:
    nltk.download('punkt', quiet=True)
    nltk.download('wordnet', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
    logger.info("NLTK Daten erfolgreich heruntergeladen")
except Exception as e:
    logger.error(f"Fehler beim Herunterladen der NLTK-Daten: {e}")

# Laden der Umgebungsvariablen
try:
    load_dotenv()
    logger.info("Umgebungsvariablen erfolgreich geladen")
except Exception as e:
    logger.error(f"Fehler beim Laden der Umgebungsvariablen: {e}")

## T0.1 Grundlegende Konfiguration
Definition der Projektkonfiguration und globaler Variablen

In [ ]:
import os
import logging
from typing import Dict, Any

class Config:
    # Modell-Parameter
    MODEL_NAME: str = "gpt2"  # Beispiel-Modell
    MAX_LENGTH: int = 128
    BATCH_SIZE: int = 16

    # Pfade
    DATA_DIR: str = "data"
    MODEL_DIR: str = "models"
    LOG_DIR: str = "logs"

    # Training
    LEARNING_RATE: float = 2e-5
    EPOCHS: int = 3

    @classmethod
    def validate_config(cls) -> Dict[str, Any]:
        """Überprüft die Konfigurationswerte auf Gültigkeit"""
        config_status = {}

        # Überprüfe numerische Werte
        if cls.MAX_LENGTH <= 0:
            config_status["MAX_LENGTH"] = "Muss größer als 0 sein"
        if cls.BATCH_SIZE <= 0:
            config_status["BATCH_SIZE"] = "Muss größer als 0 sein"
        if cls.LEARNING_RATE <= 0:
            config_status["LEARNING_RATE"] = "Muss größer als 0 sein"
        if cls.EPOCHS <= 0:
            config_status["EPOCHS"] = "Muss größer als 0 sein"

        return config_status

    @classmethod
    def create_directories(cls) -> None:
        """Erstellt die notwendigen Verzeichnisse mit Fehlerbehandlung"""
        for dir_path in [cls.DATA_DIR, cls.MODEL_DIR, cls.LOG_DIR]:
            try:
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)
                    logger.info(f"Verzeichnis erstellt: {dir_path}")
                else:
                    logger.info(f"Verzeichnis existiert bereits: {dir_path}")
            except Exception as e:
                logger.error(f"Fehler beim Erstellen des Verzeichnisses {dir_path}: {e}")
                raise

# Konfiguration überprüfen und Verzeichnisse erstellen
try:
    invalid_configs = Config.validate_config()
    if invalid_configs:
        for key, error in invalid_configs.items():
            logger.error(f"Ungültige Konfiguration für {key}: {error}")
        raise ValueError("Ungültige Konfigurationswerte gefunden")

    Config.create_directories()
    logger.info("Konfiguration erfolgreich initialisiert")
except Exception as e:
    logger.error(f"Fehler bei der Konfigurationsinitalisierung: {e}")
    raise

## T0.2 Datenstruktur-Definitionen
Definition der grundlegenden Klassen und Datenstrukturen für den Chatbot

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Optional
from datetime import datetime
import pandas as pd

@dataclass
class Message:
    text: str
    sender: str
    timestamp: datetime = None

    def __post_init__(self):
        if not isinstance(self.text, str):
            raise TypeError("text muss ein String sein")
        if not isinstance(self.sender, str):
            raise TypeError("sender muss ein String sein")
        if not self.text.strip():
            raise ValueError("text darf nicht leer sein")
        self.timestamp = self.timestamp or pd.Timestamp.now()

class Conversation:
    def __init__(self):
        self.messages: List[Message] = []
        self.context: Dict = {}

    def add_message(self, message: Message) -> None:
        """Fügt eine neue Nachricht zur Konversation hinzu"""
        if not isinstance(message, Message):
            raise TypeError("message muss vom Typ Message sein")
        self.messages.append(message)

    def get_history(self) -> List[Message]:
        """Gibt die Konversationshistorie zurück"""
        return self.messages.copy()  # Erstelle eine Kopie für die Sicherheit

class ChatbotMemory:
    def __init__(self):
        self.conversations: Dict[str, Conversation] = {}
        self.global_context: Dict = {}

    def start_conversation(self, conversation_id: str) -> None:
        """Startet eine neue Konversation"""
        if not isinstance(conversation_id, str):
            raise TypeError("conversation_id muss ein String sein")
        if conversation_id in self.conversations:
            logger.warning(f"Konversation {conversation_id} existiert bereits")
        self.conversations[conversation_id] = Conversation()

    def add_to_conversation(self, conversation_id: str, message: Message) -> None:
        """Fügt eine Nachricht zu einer bestehenden Konversation hinzu"""
        if not isinstance(conversation_id, str):
            raise TypeError("conversation_id muss ein String sein")
        if not isinstance(message, Message):
            raise TypeError("message muss vom Typ Message sein")

        if conversation_id not in self.conversations:
            logger.info(f"Erstelle neue Konversation für ID: {conversation_id}")
            self.start_conversation(conversation_id)
        self.conversations[conversation_id].add_message(message)

    def get_conversation(self, conversation_id: str) -> Optional[Conversation]:
        """Gibt eine Konversation zurück, wenn sie existiert"""
        return self.conversations.get(conversation_id)

# Test der implementierten Klassen
try:
    # Beispiel-Verwendung
    memory = ChatbotMemory()
    msg = Message("Hallo!", "Benutzer")
    memory.add_to_conversation("test_id", msg)
    logger.info("Klassen erfolgreich getestet")
except Exception as e:
    logger.error(f"Fehler beim Testen der Klassen: {e}")
    raise

## T0.3 Hilfsfunktionen
Allgemeine Hilfsfunktionen für Textverarbeitung und Datenverwaltung

In [ ]:
def preprocess_text(text):
    """
    Grundlegende Textvorverarbeitung
    """
    text = text.lower().strip()
    return text

def load_training_data(file_path):
    """
    Laden und Vorverarbeiten der Trainingsdaten
    """
    try:
        data = pd.read_csv(file_path)
        logger.info(f"Daten erfolgreich geladen: {len(data)} Einträge")
        return data
    except Exception as e:
        logger.error(f"Fehler beim Laden der Daten: {e}")
        return None

def save_model(model, path):
    """
    Speichern des trainierten Modells
    """
    try:
        torch.save(model.state_dict(), path)
        logger.info(f"Modell gespeichert unter: {path}")
    except Exception as e:
        logger.error(f"Fehler beim Speichern des Modells: {e}")

## T0.4 Evaluierungsmetriken
Definition der Metriken zur Bewertung des Chatbots

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

class ChatbotEvaluator:
    @staticmethod
    def calculate_response_accuracy(predicted_responses, true_responses):
        """
        Berechnet die Genauigkeit der Chatbot-Antworten
        """
        return accuracy_score(true_responses, predicted_responses)

    @staticmethod
    def calculate_metrics(predicted_responses, true_responses):
        """
        Berechnet verschiedene Evaluierungsmetriken
        """
        precision, recall, f1, _ = precision_recall_fscore_support(
            true_responses,
            predicted_responses,
            average='weighted'
        )
        return {
            'precision': precision,
            'recall': recall,
            'f1': f1
        }